In [5]:
import pandas as pd
import sys

sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [6]:
def read_personnel_and_employment_hist():
    dfa = pd.read_csv("../../../data/GA/2022-09-27/employment-history.csv")
    return dfa


In [7]:
dfa = read_personnel_and_employment_hist()


dfa = dfa.rename(columns={" NAME": "NAME", " START DATE": "hire_date", " END DATE ":"left_date", "OKEY": "OKEY_KEY"})

def split_names(df):
    names = df.NAME.str.lower().str.strip().str.extract(r"(\w+) (\w+) ?(.+)")

    df.loc[:, "last_name"] = names[0]
    df.loc[:, "middle_name"] = names[2]
    df.loc[:, "first_name"] = names[1]

    return df 

dfa = dfa.pipe(split_names)

dfa = dfa.pipe(gen_uid, ["first_name", "last_name", "middle_name"])
dfa


,Unnamed: 0,OKEY_KEY,NAME,AGENCY,RANK,STATUS,hire_date,left_date,last_name,middle_name,first_name,uid
0,NaN,O246465,AAMIR WISHAH,G1928 ELBERT COUNTY 911,COMM. OFFICER,Actively Employed in Law Enforcement,2020-06-01,0000-00-00,aamir,h,wisha,51a6f0fb2f36862484f36ef451b34b8b
1,NaN,O095227,AANERUD DAMON H,G1161 CHATHAM COUNTY SHERIFFS OFFICE,JAILOR,Voluntary Resignation,1999-04-19,2000-10-07,aanerud,h,damon,2bb02fa4ee7018ff2791f86f3c5b13d4
2,NaN,O095227,AANERUD DAMON H,G1244 SAVANNAH POLICE DEPARTMENT,PEACE OFFICER,Voluntary Resignation,2000-10-09,2001-03-30,aanerud,h,damon,2bb02fa4ee7018ff2791f86f3c5b13d4
3,NaN,O095227,AANERUD DAMON H,G1177 POOLER POLICE DEPARTMENT,CORPORAL,Rank Change - Promotion,2001-03-10,2009-09-27,aanerud,h,damon,2bb02fa4ee7018ff2791f86f3c5b13d4
4,NaN,O095227,AANERUD DAMON H,G1177 POOLER POLICE DEPARTMENT,SERGEANT,Actively Employed in Law Enforcement,2009-09-28,0000-00-00,aanerud,h,damon,2bb02fa4ee7018ff2791f86f3c5b13d4
...,...,...,...,...,...,...,...,...,...,...,...,...
139655,NaN,O136250,ZYDONYK ASHBY L,G1587 NEWINGTON POLICE DEPARTMENT,CAPTAIN,Rank Change - Promotion,2020-01-20,2021-01-01,zydonyk,l,ashby,233e1b2b25d037092b1421f9d3ce39c1
139656,NaN,O136250,ZYDONYK ASHBY L,G1587 NEWINGTON POLICE DEPARTMENT,CHIEF,Actively Employed in Law Enforcement,2021-01-01,0000-00-00,zydonyk,l,ashby,233e1b2b25d037092b1421f9d3ce39c1
139657,NaN,O110791,ZYGAJ STEPHEN M,G1505 ATLANTA POLICE DEPARTMENT,PEACE OFFICER,Rank Change - Promotion,2001-12-18,2020-01-08,zygaj,m,stephen,c8e8610188d437ae1de81db4a22c22cc
139658,NaN,O110791,ZYGAJ STEPHEN M,G1505 ATLANTA POLICE DEPARTMENT,CAPTAIN,Rank Change - Promotion,2020-01-09,2022-04-13,zygaj,m,stephen,c8e8610188d437ae1de81db4a22c22cc


In [ ]:
dfb = pd.read_csv("../../../data/GA/2022-09-27/officer_data-20231220144255.csv", encoding="latin1")

dfb = dfb.rename(columns={" SEX": "sex", " RACE ": "race", 
                              " YOB": "year_of_birth", " MIDDLE": "middle_name",
                              " LAST NAME": "last_name", " FIRST NAME": "first_name"})

dfb
    
dfb = dfb.pipe(clean_names, ["first_name", "last_name", "middle_name"])

    
dfb = dfb.pipe(gen_uid, ["first_name", "last_name", "middle_name"])

    
dfb = dfb[["year_of_birth", "race", "sex", "uid"]]

In [8]:
df = pd.merge(dfa, dfb, on="uid", how="left")

df["state"] = "GA"


df.sort_values("OKEY_KEY")

df.loc[:, "left_date"] = df.left_date.str.replace(r"0000-00-00", "", regex=True)

df.loc[:, "hire_date"] = df.hire_date.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1", regex=True)
df.loc[:, "left_date"] = df.left_date.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1", regex=True)

df = df[~((df.left_date.fillna("") == ""))]

df.left_date.unique()


df = df[df.duplicated(subset=["OKEY_KEY"], keep=False)]

df = df.rename(columns={" AGENCY": "agency"})

df = df.pipe(gen_uid, ["first_name", "middle_name", "last_name", "agency"])

df

,OKEY,last_name,first_name,middle_name,SUFFIX,year_of_birth,sex,race,uid
0,O246465,aamir,wishah,,,1996,,NaN,94ef64ccb665a30948af6ebd15d5a152
1,O095227,aanerud,damon,h,,1972,,NaN,2bb02fa4ee7018ff2791f86f3c5b13d4
2,O245762,aaron,brittany,denise,,1985,,NaN,39a842f021d4ce7703457a78246b1874
3,O091199,aaron,edna,m,,1957,,NaN,2447c5fb0829c0dc64b5a7712f57f121
4,O251551,aaron,hannah,elizabeth,,2000,,NaN,2024b5b3c9b08c666d0d3356358f7ac9
...,...,...,...,...,...,...,...,...,...
50155,O220418,zunun,jose,heriberto,,1992,,NaN,ebfa02ba1cb05c167cac615223060257
50156,O249398,zurkee,sophia,ann,,1996,,NaN,ecc7638929d615744cedb02b33d1d64c
50157,O133176,zwierko,michael,r,,1981,,NaN,9a386bf00d000af64e4ebd126083ef40
50158,O136250,zydonyk,ashby,l,,1985,,NaN,233e1b2b25d037092b1421f9d3ce39c1
